In [1]:
print("yea")

yea


In [6]:
# !pip install pythainlp
# !pip install wordfreq
# !pip install mecab
# !apt-get install mecab mecab-ipadic-utf8 libmecab-dev swig
# !pip install mecab-python3
# !pip install mecab-ko
# !pip install ipadic
# !pip install git+https://github.com/TalkBank/TBDBpy.git

import nltk
nltk.download('punkt')

import codecs
import random
import shutil
import ssl
import time
import zipfile
from string import punctuation
from io import StringIO

import os
import re
import requests
import urllib3
from lxml import etree
import nltk
from nltk.tokenize import word_tokenize as nltk_tokenize
from pythainlp import word_tokenize as thai_tokenize
from requests.exceptions import Timeout, ConnectionError
from urllib3.exceptions import ReadTimeoutError
from wordfreq import tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer

import MeCab
import mecab_ko_dic

import tbdb

lang_dic = { 
    'en': ['English_eng', 'eng', 'English', 'English', 'Latn'],

    'es': ['Spanish_spa', 'spa', 'Spanish', 'Spanish', 'Latn'],
    'eu': ['Basque_eus', 'eus', 'Basque', 'Basque', 'Latn'],
    'fr': ['French_fra', 'fra', 'French', 'French', 'Latn'],
    'de': ['German_deu', 'deu', 'German', 'German', 'Latn'],
    'el': ['Greek_Modern_ell', 'ell', 'Greek (Modern)', 'Modern Greek', 'Grek'],
    'he': ['Hebrew_Modern_heb', 'heb', 'Hebrew (Modern)', 'Modern Hebrew', 'Hebr'],
    'id': ['Indonesian_ind', 'ind', 'Indonesian', 'Indonesian', 'Latn'],
    'ja': ['Japanese_jpn', 'jpn', 'Japanese', 'Japanese', 'Jpan'],
    'ko': ['Korean_kor', 'kor', 'Korean', 'Korean', 'Kore'],  
    'fa': ['Persian_pes', 'pes', 'Persian', 'Western Farsi', 'Arab'],
    'ru': ['Russian_rus', 'rus', 'Russian', 'Russian', 'Cyrl'],
    'th': ['Thai_tha', 'tha', 'Thai', 'Thai', 'Thai'],
    'tl': ['Tagalog_tgl', 'tgl', 'Tagalog', 'Tagalog', 'Latn'],
    'tr': ['Turkish_tur', 'tur', 'Turkish', 'Turkish', 'Latn']
    
}

report = codecs.open('../report_conversationTalkBank.csv', 'a', 'utf-8')
report.write('folder_name;tokens;txt_files\n')

resource = ''
copyright_short = ''
copyright_long = ''
duplicate_text_set = set()

total_tokens = 0
first_counter = 0
current_counter = 0



# get content from link
def get_content(link):
  response = requests.get(link)
  content = response.text
  # print(content)
  return content

# removes lines that start with @ or %
def remove_metadata_and_comments(text):
  lines = text.split('\n')
  filtered_lines = filter(lambda line: line.startswith('*'), lines)
  result = '\n'.join(filtered_lines)
  return result


# get date when resouce was created
def parse_date(content):
  pattern = r"@Date:\s+\d{1,2}-\w{3}-(\d{4})"
  match = re.search(pattern, content)
  if match:
      year = match.group(1)
  else:
      year = 'NA'
  return year


# read text line by line and remove everything before first : (speaker name)
def remove_speakers_names(text):
  lines = text.split('\n')
  result = []

  for line in lines:
      result.append(line.split(':', 1)[-1].strip())

  result_text = '\n'.join(result)
  return result_text


# filtering (.), [.*]
def filter_text(text):
  text = text.replace("(.)", "")
  text = re.sub(r'\[.*?\]', '', text)
  stripped_lines = [line.strip() for line in text.split('\n')]
  text = '\n'.join(stripped_lines)

  return text

def extract50K(text, lang):
    tokens_extracted = 0
    text = StringIO(text)
    text50K = ''
    remaining_text = ''
    while(True):
        nl = text.readline()
        if nl != '':
            tokens_extracted += count_tokens(nl, lang)[0]
            if(tokens_extracted < 50000):
                text50K += nl
            else:
                remaining_text += nl
        else:
            break;
    return text50K, remaining_text


def define_line_start(arr_text):
    random_range = [0]
    for i in range(int(len(arr_text)/2)):
        if arr_text[i][-1] in ['?','.']:
            char_start = i
            random_range.append(char_start)
    char_start = random.choice(random_range)
    return char_start

# Random starting point
def starting_point(arr_text, tokens, lang):
    if tokens > 100001:
        line_start = define_line_start(arr_text)
        return line_start
    else:
        return 0

def write_50k_to_file(fname,lang_dic,lang,year,resource,link,text,isWhole):
    f1 = codecs.open(fname, 'w', 'utf-8')
    meta = '''# language_name_wals:	''' + lang_dic[lang][2] + '''
# language_name_glotto:	''' + lang_dic[lang][3] + '''
# iso639_3:	''' + lang_dic[lang][1] + '''
# year_composed:	NA
# year_published:	''' + year + '''
# mode:	written
# genre_broad:	conversation
# genre_narrow:	NA
# writing_system:	''' + lang_dic[lang][4] + '''
# special_characters:	NA
# short_description:	''' + resource + '''
# source:	''' + link + '''
# copyright_short:	''' + copyright_short + '''
# copyright_long:	''' + copyright_long + '''
# sample_type:	''' + isWhole + '''
# comments:	NA
'''
    meta = re.sub('\t{2,}', '\t', meta)
    meta = re.sub(' {2,}', ' ', meta)
    f1.write(meta)

    text = re.sub('\t+', ' ', text)
    text = re.sub(' {2,}', ' ', text)

    f1.write(text[:-1])
    f1.close()

# Root path
def get_root(lang):
    pathToZip = os.path.join('..','Corpus',lang_dic[lang][0], 'conversation' )

    if not os.path.exists(pathToZip):
        os.makedirs(pathToZip)
    return pathToZip

# Generate file name
def generate_fname(lang, current_counter):
    path = get_root(lang)
    max_counter = 0
    search_fcounter = re.compile(lang_dic[lang][1] + '_con_' + '([0-9]+)(\\.txt)?')
    for root, dirs, files in os.walk(path):
        for fname in files:
            fcounter = re.search(search_fcounter, fname)

            if fcounter is not None:
                if int(fcounter.group(1)) > max_counter:
                    current_counter = fcounter.group(1)
                    max_counter = int(current_counter)
            else:
                current_counter = 0
    if max_counter > 0:
        current_counter = max_counter
    fname = lang_dic[lang][1] + '_con_' + str(int(current_counter) + 1) + '.txt'
    return os.path.join(get_root(lang) ,fname), current_counter

# Sample
def sample(link, text, year, tokens, lang, current_counter, total_tokens,tokens_arr):
    fname = ''
    if int(tokens) <= 50000 and int(tokens) > 0:
        fname, current_counter = generate_fname(lang, current_counter)
        write_50k_to_file(fname,lang_dic,lang,year,resource,link,text,'whole')
        total_tokens += int(tokens)

    if(tokens>50000):  
        whole_file_covered = 0
        while not whole_file_covered:
            starting_place = starting_point(text, tokens, lang)
            if (starting_place!=0):
                starting_place+=1
                
            left, right = text[:starting_place], text[starting_place:]
            text_with_only_50k, remaining_text = extract50K(right, lang)
            remaining_text = left + remaining_text          

            fname, current_counter = generate_fname(lang, current_counter)
            write_50k_to_file(fname,lang_dic,lang,year,resource,link,text_with_only_50k,'part')
            tokens = count_tokens(text_with_only_50k, lang)[0]
            total_tokens += tokens
            text = remaining_text
            tokens_arr = count_tokens(text, lang)[1]
            token_cnt, t = count_tokens(text, lang)

            if(token_cnt < 100):
                whole_file_covered = 1
                   
    return total_tokens, current_counter, fname, tokens

# Tokenization
def count_tokens(text, lang):
    if(lang =='zh_CN'):
        lang = 'zh'
    if(lang == 'en_GB'):
        lang = 'en'
    if lang in ['en', 'fi', 'fr', 'de', 'el', 'he', 'hi', 'id', 'ja', 'ko', 'zh','fa', 'ru', 'es', 'tr']:
        tokens = tokenize(text, lang)
    elif lang == 'th':
        tokens = thai_tokenize(text)
    else:
        tokens = nltk_tokenize(text)

    tokens_tmp = [word.lower() for word in tokens
              if not (word in punctuation and not word.isdigit() and not word.isspace())]

    return len(tokens_tmp),tokens

# Retrieve text from document and filter it
def get_text(lang, checkForDuplicates, content):
    if(not content):
      return False

    text_for_checking_duplicates = ''
    num_line = 0

    year = parse_date(content)
    content = remove_metadata_and_comments(content)
    content = remove_speakers_names(content)
    text = filter_text(content)

    text_for_checking_duplicates = tuple(text.splitlines()[:20])

    tokens, tokens_arr = count_tokens(text, lang)

    if(checkForDuplicates):
        if(text_for_checking_duplicates in duplicate_text_set):
            return False
        else:
            duplicate_text_set.add(text_for_checking_duplicates)
        
    return text, year, tokens, tokens_arr

def total_tokens_not_reached(total_tokens):
    if ((total_tokens > 5000000)):
        return False
    else:
        return True

def search_xmls(lang, total_tokens,current_counter, linkToFiles):
    tokens_per_resource_per_language = 0
    for row in linkToFiles['data']:
        path = row[linkToFiles['colHeadings'].index('path')]

        link = createUrl(path)
        text = get_content(link)

        if total_tokens_not_reached(total_tokens):
            if get_text(lang, True, text) is not False:
                text, year, tokens, tokens_arr = get_text(lang, False, text)

                total_tokens, current_counter, fname, current_tokens = sample(link, text, year, tokens, lang,
                                                              current_counter, total_tokens, tokens_arr)
                tokens_per_resource_per_language += current_tokens
                print("Saved file", fname)

            else:
                print('Skipped file due to parsing error or duplicate file')
        else:
            return total_tokens, current_counter

    return total_tokens, current_counter


# url to one text file
def createUrl(path):
#     todo change when needed versions: homebank, class,samtale, ca, phon
  new_path = 'https://ca.talkbank.org/data-orig/' + path.split('/', 1)[1] + '.cha'
  print(new_path)
  return new_path

def skip_resource_language(resource, language):
    return False


def main():
    print("STARTING....")
    total_tokens_for_all_languages = 0
    
    for lang in lang_dic:
        current_counter = 0
        #total_tokens for single language for all resources
        total_tokens = 0
#         todo change
# versions: homebank, class,samtale,ca,phon
        for curr_resource in ['ca']:
            if(skip_resource_language(curr_resource,lang)):
                continue
            global resource
            resource = curr_resource
            global copyright_short
#             //todo change  versions: homebank, class,samtale,ca,phon
            copyright_short = 'https://sla.talkbank.org/TBB/ca'
            global copyright_long
#         todo change versions:
# CABank: MacWhinney, B., & Wagner, J. (2010). Transcribing, searching and data sharing: The CLAN software and the TalkBank data repository. Gesprachsforschung, 11, 154-173.
# SamtaleBank MacWhinney, B. (2019). Understanding spoken language through TalkBank. Behavior Research Methods. doi:10.3758/s13428-018-1174-9
# ClassBank: MacWhinney, B. (2007). A transcript-video database for collaborative commentary in the Learning Sciences. In R. Goldman, R. Pea, B. Barron, & S. Derry (Eds.), Video research in the learning sciences (pp. 537-546). Mahwah, NJ: Lawrence Erlbaum Associates.
# Phon and PhonBank: Rose, Y., & MacWhinney, B. (2014). The PhonBank Project: Data and software-assisted methods for the study of phonology and phonological development. In Durand J., Gut U., & Kristoffersen, G. (Eds.), The Oxford handbook of corpus phonology (pp. 380-401). Oxford, UK: Oxford University Press. Note: Please also acknowledge this grant support for PhonBank -- NIH-NICHHD RO1-HD051698.
# HomeBank: VanDam, M., Warlaumont, A. S., Bergelson, E., Cristia, A., Soderstrom, M., De Palma, P., & MacWhinney, B. (2016). HomeBank: An online repository of daylong child-centered audio recordings. Seminars in Speech and Language, 37(2), 128-142. doi:10.1055/s-0036-1580745 Note: Please also acknowledge this grant support for HomeBank -- NSF SBE RIDIR Grants 1539129, 1539133, and 1539010.

            copyright_long = 'CABank: MacWhinney, B., & Wagner, J. (2010). Transcribing, searching and data sharing: The CLAN software and the TalkBank data repository. Gesprachsforschung, 11, 154-173.'

            global duplicate_text_set
            duplicate_text_set = set()
    
            print("Language: ",lang_dic[lang][2])
            #get all files for language
#             todo change!!  versions: homebank, class,samtale,ca,phon
            resources = tbdb.getTranscripts( {"corpusName": "ca", "lang": [lang_dic[lang][1]]} )
            #remove files that contain more than one language
            new_data = [row for row in resources['data'] if ',' not in row[2]]
            resources['data'] = new_data

            total_tokens, current_counter = search_xmls(lang, total_tokens,current_counter, resources)
            
        total_tokens_for_all_languages += total_tokens
        report.write(lang_dic[lang][0] + ';' + str(total_tokens) + ';' + str(current_counter+1) + '\n')
    # print("Total tokens collected for all languages: ", total_tokens_for_all_languages)
    report.close()
    print("END...")


if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\i.veljkovic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


STARTING....
Language:  English
https://ca.talkbank.org/data-orig/Bradford/01.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10117.txt
https://ca.talkbank.org/data-orig/Bradford/02.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10118.txt
https://ca.talkbank.org/data-orig/Bradford/03.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10119.txt
https://ca.talkbank.org/data-orig/Bradford/04.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10120.txt
https://ca.talkbank.org/data-orig/Bradford/05.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10121.txt
https://ca.talkbank.org/data-orig/Bradford/06.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10122.txt
https://ca.talkbank.org/data-orig/Bradford/07.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10123.txt
https://ca.talkbank.org/data-orig/Bradford/08.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10124.txt
https://ca.talkbank.org/data-orig/Bradford/09.cha
Saved 

Saved file ..\Corpus\English_eng\conversation\eng_con_10189.txt
https://ca.talkbank.org/data-orig/Bradford/74.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10190.txt
https://ca.talkbank.org/data-orig/Bradford/75.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10191.txt
https://ca.talkbank.org/data-orig/Bradford/76.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10192.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KB0RE004.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10193.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KB0RE006.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10194.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KB1RE000.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10195.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KB1RE001.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10196.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KB1RE005.cha
Saved file ..\Corpus\English

Saved file ..\Corpus\English_eng\conversation\eng_con_10255.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBHRE00P.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10256.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBHRE00R.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10257.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBHRE00S.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10258.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBHRE00T.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10259.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBJRE005.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10260.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBJRE00B.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10261.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBJRE00P.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10262.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KBKRE

Saved file ..\Corpus\English_eng\conversation\eng_con_10321.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KCGRE00G.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10322.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KCHRE000.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10323.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KCHRE004.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10324.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KCHRE00B.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10325.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KCHRE00C.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10326.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KE0RE006.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10327.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KE0RE00F.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10328.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KE0RE

Saved file ..\Corpus\English_eng\conversation\eng_con_10387.txt
https://ca.talkbank.org/data-orig/CABNC/0missing/KE6RE01T.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10388.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE000.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10389.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE001.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10390.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE002.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10391.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE003.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10392.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE005.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10393.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE007.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10394.txt
https://ca.talkbank.org/data-orig/CABNC/KB0/KB0RE008.cha
Saved file ..\Corpus\Englis

Saved file ..\Corpus\English_eng\conversation\eng_con_10455.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE001.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10456.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE002.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10457.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE003.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10458.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE004.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10459.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE005.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10460.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE006.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10461.txt
https://ca.talkbank.org/data-orig/CABNC/KB6/KB6RE007.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10462.txt
https://ca.talkbank.org/data-orig/CABNC/KB7/KB7RE000.cha
Saved file ..\Corpus\English_eng

Saved file ..\Corpus\English_eng\conversation\eng_con_10523.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE004.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10524.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE006.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10525.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE007.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10526.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE008.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10527.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE009.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10528.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE00A.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10529.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE00B.cha
Saved file ..\Corpus\English_eng\conversation\eng_con_10530.txt
https://ca.talkbank.org/data-orig/CABNC/KB8/KB8RE00C.cha
Saved file ..\Corpus\English_eng

KeyboardInterrupt: 